In [1]:
import os
import pandas as pd
import jupyter_black

jupyter_black.load()

current_dir = os.path.dirname(os.path.abspath("__file__"))

### Set file path to datasets

In [2]:
fortune_path = os.path.join(current_dir, "data", "fortune1000.csv")

In [3]:
fortune = pd.read_csv(fortune_path, index_col="Rank")
fortune["Sector"] = fortune["Sector"].astype("category")
fortune["Industry"] = fortune["Industry"].astype("category")
print(fortune.info())
display(fortune.head(3))

sectors = fortune.groupby("Sector")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Company    1000 non-null   object  
 1   Sector     1000 non-null   category
 2   Industry   1000 non-null   category
 3   Location   1000 non-null   object  
 4   Revenue    1000 non-null   int64   
 5   Profits    1000 non-null   int64   
 6   Employees  1000 non-null   int64   
dtypes: category(2), int64(3), object(2)
memory usage: 52.2+ KB
None


,Company,Sector,Industry,Location,Revenue,Profits,Employees
Rank,,,,,,,
1,Walmart,Retailing,General Merchandisers,"Bentonville, AR",482130,14694,2300000
2,Exxon Mobil,Energy,Petroleum Refining,"Irving, TX",246204,16150,75600
3,Apple,Technology,"Computers, Office Equipment","Cupertino, CA",233715,53394,110000


A pandas `groupby` object is kind of a group of pandas dataframes that are bundelled together (one for each unique value in the groupby column)

`len()`

In [4]:
print(len(sectors))
print(fortune["Sector"].nunique())

21
21


`size()`

In [5]:
print(sectors.size().sort_values(ascending=False).head(3))
print(fortune["Sector"].value_counts().head(3))

Sector
Financials    139
Energy        122
Technology    102
dtype: int64
Financials    139
Energy        122
Technology    102
Name: Sector, dtype: int64


`first()` (returns the **first** column for each group)

In [6]:
sectors.first().head(3)

,Company,Industry,Location,Revenue,Profits,Employees
Sector,,,,,,
Aerospace & Defense,Boeing,Aerospace and Defense,"Chicago, IL",96114,5176,161400
Apparel,Nike,Apparel,"Beaverton, OR",30601,3273,62600
Business Services,ManpowerGroup,Temporary Help,"Milwaukee, WI",19330,419,27000


`last()` (returns the **last** column for each group)

In [7]:
sectors.last().head(3)

,Company,Industry,Location,Revenue,Profits,Employees
Sector,,,,,,
Aerospace & Defense,Delta Tucker Holdings,Aerospace and Defense,"McLean, VA",1923,-133,12000
Apparel,Guess,Apparel,"Los Angeles, CA",2204,82,13500
Business Services,DeVry Education Group,Education,"Downers Grove, IL",1910,140,11770


The `groups` attribute returns a dictionary with the indices for each group

In [8]:
group_indices = sectors.groups
group_indices["Aerospace & Defense"]
group_indices["Technology"]
# One way to retrieve a group from the groupby object:
fortune.loc[group_indices["Technology"]].head(3)

,Company,Sector,Industry,Location,Revenue,Profits,Employees
Rank,,,,,,,
3,Apple,Technology,"Computers, Office Equipment","Cupertino, CA",233715,53394,110000
18,Amazon.com,Technology,Internet Services and Retailing,"Seattle, WA",107006,596,230800
20,HP,Technology,"Computers, Office Equipment","Palo Alto, CA",103355,4554,287000


### Retrieve a group from a GroupBy object with the `get_group()` method

In [9]:
sectors.get_group("Technology").sort_values(by="Profits", ascending=False).head(5)

,Company,Sector,Industry,Location,Revenue,Profits,Employees
Rank,,,,,,,
3,Apple,Technology,"Computers, Office Equipment","Cupertino, CA",233715,53394,110000
36,Alphabet,Technology,Internet Services and Retailing,"Mountain View, CA",74989,16348,61814
31,IBM,Technology,Information Technology Services,"Armonk, NY",82461,13190,411798
25,Microsoft,Technology,Computer Software,"Redmond, WA",93580,12193,118000
51,Intel,Technology,Semiconductors and Other Electronic Components,"Santa Clara, CA",55355,11420,107300


### Methods on the GroupBy object and DataFrame Columns

In [10]:
sectors[["Profits"]].max()
sectors[["Profits"]].min()
sectors[["Profits"]].sum().sort_values(by="Profits", ascending=False)
sectors[["Profits"]].mean().sort_values(by="Profits", ascending=False).head(5)
sectors[["Employees"]].sum().sort_values(by="Employees", ascending=False).head(3)

,Employees
Sector,
Retailing,6227629
Technology,3578949
Financials,3359948


### Grouping multiple columns

In [11]:
sectors = fortune.groupby(["Sector", "Industry"])
sectors.size()
sectors[["Revenue"]].sum()

Revenue
Sector              Industry                                              
Aerospace & Defense Advertising, marketing                               0
                    Aerospace and Defense                           357940
                    Airlines                                             0
                    Apparel                                              0
                    Automotive Retailing, Services                       0
...                                                                    ...
Wholesalers         Waste Management                                     0
                    Wholesalers: Diversified                        176138
                    Wholesalers: Electronics and Office Equipment   147906
                    Wholesalers: Food and Grocery                   111774
                    Wholesalers: Health Care                             0

[1533 rows x 1 columns]

### The `agg()` method

In [12]:
sectors = fortune.groupby("Sector")
sectors.agg({"Revenue": "sum"}).head(3)
sectors.agg({"Revenue": "sum", "Profits": "mean", "Employees": "std"}).head(3)
sectors[["Revenue", "Profits", "Employees"]].agg(["size", "sum", "mean"]).head(3)
sectors.agg({"Revenue": ["sum", "mean"], "Profits": "mean", "Employees": "std"}).head(3)

Revenue                    Profits     Employees
                        sum          mean         mean           std
Sector                                                              
Aerospace & Defense  357940  17897.000000  1437.100000  55389.486777
Apparel               95968   6397.866667   549.066667  21833.228179
Business Services    272195   5337.156863   553.470588  37717.982486

### Iterating through the groups

In [13]:
it = 0
for sector, data in sectors:
    print(sector)
    display(data.head(3))
    it += 1
    if it >= 2:
        break

Aerospace & Defense


,Company,Sector,Industry,Location,Revenue,Profits,Employees
Rank,,,,,,,
24,Boeing,Aerospace & Defense,Aerospace and Defense,"Chicago, IL",96114,5176,161400
45,United Technologies,Aerospace & Defense,Aerospace and Defense,"Farmington, CT",61047,7608,197200
60,Lockheed Martin,Aerospace & Defense,Aerospace and Defense,"Bethesda, MD",46132,3605,126000


Apparel


,Company,Sector,Industry,Location,Revenue,Profits,Employees
Rank,,,,,,,
91,Nike,Apparel,Apparel,"Beaverton, OR",30601,3273,62600
231,VF,Apparel,Apparel,"Greensboro, NC",12377,1232,64000
340,PVH,Apparel,Apparel,"New York, NY",8020,572,26200


Getting the row with the highest profits for each group

In [14]:
df = pd.DataFrame(columns=fortune.columns)

for sector, data in sectors:
    df = pd.concat([df, data.nlargest(1, "Profits")])

display(df.head(3))

,Company,Sector,Industry,Location,Revenue,Profits,Employees
45,United Technologies,Aerospace & Defense,Aerospace and Defense,"Farmington, CT",61047,7608,197200
91,Nike,Apparel,Apparel,"Beaverton, OR",30601,3273,62600
204,Visa,Business Services,Financial Data Services,"Foster City, CA",13880,6328,11300


In [15]:
df = pd.DataFrame(columns=fortune.columns)

cities = fortune.groupby("Location")

for city, data in cities:
    df = pd.concat([df, data.nlargest(1, "Profits")])

display(df.sort_values(by="Profits", ascending=False).head(3))

,Company,Sector,Industry,Location,Revenue,Profits,Employees
3,Apple,Technology,"Computers, Office Equipment","Cupertino, CA",233715,53394,110000
23,J.P. Morgan Chase,Financials,Commercial Banks,"New York, NY",101006,24442,234598
4,Berkshire Hathaway,Financials,Insurance: Property and Casualty (Stock),"Omaha, NE",210821,24083,331000
